<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #7: Transfer Object Detection models to Zenodo</h1>
<h3 align="right">Written by KSO Team</h3>

# Set up KSO requirements

In [ ]:
# @title <font size="5">↓ ឵឵<i>Install kso-object-detection and its requirements</font> { vertical-output: true }
from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules https://github.com/ocean-data-factory-sweden/koster_yolov4.git
    !pip install -q --upgrade pip
    !pip install -qr koster_yolov4/requirements.txt

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Replace upsampling script with custom version
    os.chdir("koster_yolov4/tutorials")
    !mv ../src/upsampling.py /usr/local/lib/python3.7/dist-packages/torch/nn/modules/upsampling.py

    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/nn_matching.py ../yolov5_tracker/strong_sort/sort/nn_matching.py
    !cp ../src/track.py ../yolov5_tracker/track.py

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget

    print("All packages are installed and ready to go!")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False
    import sys
    import pkgutil

    if pkgutil.find_loader("torch") is not None:
        !pip install -q --upgrade pip
        !pip install -q torch==1.8.0 torchvision==0.9.0
    # Replace nearest neighbours script with custom version (due to relative path issue)
    !cp ../src/nn_matching.py ../yolov5_tracker/strong_sort/sort/nn_matching.py
    !cp ../src/track.py ../yolov5_tracker/track.py
    # Ensure widgets are shown properly
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    clear_output()
    print("Running locally... you're good to go!")


#######Import Python packages########

# Set the directory of the libraries
import sys, os
from pathlib import Path

# Enables testing changes in utils
%load_ext autoreload
%autoreload 2

# Specify the path of the tutorials
sys.path.append("..")

# Set to display dataframes as interactive tables
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t6_utils as t6
import kso_utils.t7_utils as t7
import kso_utils.zenodo_utils as zenodo_utils
import wandb

# Model-specific imports
import yolov5.train as train
import yolov5.val as val
import yolov5.detect as detect

clear_output()
print("Packages loaded successfully")

# Find model and add to Zenodo

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose the model</font> { vertical-output: true }
model = t6.choose_model(project_name.value)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder to download the model</font> { vertical-output: true }
download_dir = t_utils.choose_folder(".", "downloaded model")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Download model</font> { vertical-output: true }
artifact_dir = t6.get_model(model.value, project_name.value, download_dir.value)

# ZENODO Upload

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Provide access token to your account</font> { vertical-output: true }
# It is found in API settings by going to your Zenodo profile and clicking on Applications.
ACCESS_TOKEN = ""

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Get bucket url for uploading files</font> { vertical-output: true }
depo_id, bucket_url = zenodo_utils.get_zenodo_id_bucket(ACCESS_TOKEN)

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Find archive for upload to Zenodo</font> { vertical-output: true }
archive_dir = t_utils.choose_folder(".", "archive for upload")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Add archive to upload</font> { vertical-output: true }
zenodo_utils.add_file_to_zenodo_upload(
    ACCESS_TOKEN,
    bucket_url,
    [f for f in Path(artifact_dir).iterdir() if f.is_file() and ".pt" in str(f)][-1],
)

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Indicate database title</font> { vertical-output: true }
upload_title = t7.choose_text("title")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Write database description</font> { vertical-output: true }
upload_description = t7.choose_text("description")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Indicate database authors together with  their organisations</font> { vertical-output: true }
authors = t7.WidgetMaker()
authors

In [ ]:
authors.author_dict

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Add dataset metadata to upload</font> { vertical-output: true }
zenodo_utils.add_metadata_zenodo_upload(
    ACCESS_TOKEN,
    depo_id,
    upload_title.value,
    upload_description.value,
    authors.author_dict,
)

In [ ]:
# END